In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [7]:
import cv2
import numpy as np
from ultralytics import YOLO
import torch
from datetime import datetime

class AccidentDetectionSystem:
    def __init__(self, model_path="best.pt", conf_threshold=0.5, iou_threshold=0.45):
        """
        Initialize the Accident Detection System
        Args:
            model_path: Path to custom trained YOLO model
            conf_threshold: Confidence threshold for detections
            iou_threshold: IOU threshold for NMS
        """
        self.model = YOLO(model_path)
        self.conf_threshold = conf_threshold
        self.iou_threshold = iou_threshold
        
        # Define class colors (BGR format)
        self.colors = {
            0: (0, 255, 0),    # bike: green
            1: (0, 0, 255),    # bike_bike_accident: red
            2: (0, 0, 255),    # bike_object_accident: red
            3: (0, 0, 255),    # bike_person_accident: red
            4: (0, 255, 0),    # car: green
            5: (0, 0, 255),    # car_bike_accident: red
            6: (0, 0, 255),    # car_car_accident: red
            7: (0, 0, 255),    # car_object_accident: red
            8: (0, 0, 255),    # car_person_accident: red
            9: (0, 255, 0)     # person: green
        }

        # Initialize class names
        self.class_names = {
            0: 'bike',
            1: 'bike_bike_accident',
            2: 'bike_object_accident',
            3: 'bike_person_accident',
            4: 'car',
            5: 'car_bike_accident',
            6: 'car_car_accident',
            7: 'car_object_accident',
            8: 'car_person_accident',
            9: 'person'
        }

        # Initialize accident event logger
        self.accident_log = []

    def is_accident_class(self, class_id):
        """Check if the class ID represents an accident"""
        accident_classes = [1, 2, 3, 5, 6, 7, 8]  # All accident-related class IDs
        return class_id in accident_classes

    def log_accident(self, class_name, confidence, frame_time):
        """Log accident details with timestamp"""
        accident_entry = {
            'timestamp': frame_time,
            'type': class_name,
            'confidence': confidence
        }
        self.accident_log.append(accident_entry)

    def process_frame(self, frame):
        """
        Process a single frame for accident detection
        Args:
            frame: Input frame (numpy array)
        Returns:
            frame: Processed frame with annotations
            bool: Whether an accident was detected in this frame
        """
        # Get current timestamp
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        # Run YOLO inference
        results = self.model(frame, conf=self.conf_threshold, iou=self.iou_threshold)[0]
        
        # Initialize accident detected flag
        accident_detected = False
        
        # Process detections
        for box in results.boxes:
            # Get box coordinates
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            
            # Get class and confidence
            class_id = int(box.cls[0])
            confidence = float(box.conf[0])
            
            # Get color for this class
            color = self.colors[class_id]
            
            # Draw bounding box
            cv2.rectangle(frame, 
                        (int(x1), int(y1)), 
                        (int(x2), int(y2)), 
                        color, 2)
            
            # Prepare label
            label = f"{self.class_names[class_id]} {confidence:.2f}"
            
            # Add label to frame
            cv2.putText(frame, label, 
                       (int(x1), int(y1 - 10)),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
            
            # Check if this is an accident class
            if self.is_accident_class(class_id):
                accident_detected = True
                self.log_accident(self.class_names[class_id], confidence, current_time)
        
        # Add timestamp to frame
        cv2.putText(frame, current_time, 
                   (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        
        # If accident detected, add warning text
        if accident_detected:
            cv2.putText(frame, "ACCIDENT DETECTED!", 
                       (frame.shape[1]//2 - 100, 50),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        
        return frame, accident_detected

    def process_video(self, input_path, output_path):
        """
        Process video file for accident detection
        Args:
            input_path: Path to input video
            output_path: Path to save output video
        """
        # Open video capture
        cap = cv2.VideoCapture(input_path)
        
        # Get video properties
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        
        # Initialize video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        
        # Initialize frame counter
        frame_count = 0
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            frame_count += 1
            
            # Process frame
            processed_frame, accident_detected = self.process_frame(frame)
            
            # Write frame
            out.write(processed_frame)
            
            # Display frame (optional)
            cv2.imshow('Accident Detection', processed_frame)
            
            # Break loop if 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        # Release resources
        cap.release()
        out.release()
        cv2.destroyAllWindows()
        
        # Save accident log
        self.save_accident_log("accident_log.txt")

    def save_accident_log(self, log_path):
        """Save accident log to file"""
        with open(log_path, 'w') as f:
            f.write("Accident Detection Log\n")
            f.write("=====================\n\n")
            for entry in self.accident_log:
                f.write(f"Time: {entry['timestamp']}\n")
                f.write(f"Type: {entry['type']}\n")
                f.write(f"Confidence: {entry['confidence']:.2f}\n")
                f.write("---------------------\n")

def main():
    # Initialize detector
    detector = AccidentDetectionSystem(
        model_path="C:/Users/DELL/Desktop/only traffic project/Accident tracking/best.pt",
        conf_threshold=0.5,  # Adjust based on your model's performance
        iou_threshold=0.45
    )
    
    # Process video
    input_video = "C:/Users/DELL/Desktop/only traffic project/Accident tracking/video5.mp4"  # Replace with your input video path
    output_video = "C:/Users/DELL/Desktop/only traffic project/Accident tracking/output_video5.mp4"  # Replace with desired output path
    detector.process_video(input_video, output_video)

if __name__ == "__main__":
    main()


0: 288x512 1 car, 23.1ms
Speed: 0.0ms preprocess, 23.1ms inference, 0.0ms postprocess per image at shape (1, 3, 288, 512)

0: 288x512 1 car, 2.5ms
Speed: 16.5ms preprocess, 2.5ms inference, 0.0ms postprocess per image at shape (1, 3, 288, 512)

0: 288x512 1 car, 8.0ms
Speed: 1.0ms preprocess, 8.0ms inference, 1.0ms postprocess per image at shape (1, 3, 288, 512)

0: 288x512 1 car, 0.0ms
Speed: 1.2ms preprocess, 0.0ms inference, 0.0ms postprocess per image at shape (1, 3, 288, 512)

0: 288x512 1 car, 0.0ms
Speed: 0.0ms preprocess, 0.0ms inference, 12.2ms postprocess per image at shape (1, 3, 288, 512)

0: 288x512 1 car, 0.0ms
Speed: 0.0ms preprocess, 0.0ms inference, 0.0ms postprocess per image at shape (1, 3, 288, 512)

0: 288x512 1 car, 0.0ms
Speed: 0.0ms preprocess, 0.0ms inference, 0.0ms postprocess per image at shape (1, 3, 288, 512)

0: 288x512 1 car, 0.0ms
Speed: 0.0ms preprocess, 0.0ms inference, 0.0ms postprocess per image at shape (1, 3, 288, 512)

0: 288x512 1 car, 0.0ms
Spe

In [12]:
#to know what classes are there on .pt 
from ultralytics import YOLO

def print_model_classes(model_path):
    """
    Print all classes available in a YOLO model
    Args:
        model_path: Path to the .pt file
    """
    # Load the model
    model = YOLO(model_path)
    
    # Get the names dictionary
    names = model.names
    
    # Print all classes with their indices
    print("\nAvailable classes in the model:")
    print("--------------------------------")
    for idx, class_name in names.items():
        print(f"Class {idx}: {class_name}")
    
    return names

# Example usage
if __name__ == "__main__":
    # Replace with your model path
    model_path = "C:/Users/DELL/Desktop/accident detection/Accident-Detection-Model-master/runs/detect/train/weights/best.pt"
    classes = print_model_classes(model_path)

   


Available classes in the model:
--------------------------------
Class 0: Accident
